# Segmenting and Clustering Cities in the World
## Segmenting and Clustering Citiess with Foursquare information
---
**Xu Qianyi**

Data Scientist

**In thie project, we are going to examine what are the most similar city in the world for some famous cities in the world, for example, would New York be the most similar city to London, since they are both famous finacial cities.**

**Also, we will finally cluster 20 famous cities in worlds into 5 classes, and analyze the features of each cluster.**

## 1. Load location of Cities

import all required libraries

In [23]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

# !conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    openssl-1.0.2p             |       h470a237_2         3.1 MB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    altair-2.3.0               |        py36_1001         533 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.7 MB

The following NEW packages will be INSTALLED:

    altair:  2.3.0-py36_1001   conda-forge
    branca:  0.3.1-py_0        conda-forge
    folium:  0.5.0-py_0        conda-forge
    vincent: 0.4.

In [97]:
from sklearn.metrics.pairwise import cosine_similarity

First, we collect List of cities by GDP and by population from Internet, all data required for this project is saved in a csv file.

> [List of cities by GDP](https://en.wikipedia.org/wiki/List_of_cities_by_GDP)

> ![img](https://raw.githubusercontent.com/oliviaHKUST/Capstone-Project/master/Cities_by_GDP.png)

> [List of cities by population](https://www.worldatlas.com/citypops.htm)

> ![img](https://raw.githubusercontent.com/oliviaHKUST/Capstone-Project/master/Cities_by_Population.png)

load csv file as dataframe

In [18]:
df_cities = pd.read_csv('https://raw.githubusercontent.com/oliviaHKUST/Capstone-Project/master/City_Infos.csv')
# df_loc = df.rename(columns={'Postal Code' : 'PostalCode'})
print(df_cities.shape)
df_cities

(17, 6)


,City,Country,Region,GDP,Population,GDP_Div_Pop
0,Tokyo,Japan,East Asia,1893.000,38001000,49.814479
1,New York,United States,North America,1717.712,18593220,92.383783
2,Los Angeles,United States,North America,1001.000,12309530,81.319108
3,London,United Kingdom,Northern Europe,879.500,10313307,85.278175
4,Paris,France,Western Europe,850.000,10843285,78.389529
5,Chicago,United States,North America,651.222,8744835,74.469330
6,Seoul,South Korea,East Asia,635.400,9773746,65.010898
7,Osaka,Japan,East Asia,584.000,20237645,28.857113
8,Sao Paulo,Brazil,South America,582.079,21066245,27.630885
9,Rhine-Ruhr,Germany,Western Europe,573.700,10356889,55.393082


Get latitude and longitude of all cities with geocoder library

In [19]:
df_cities["Latitude"] = 0
df_cities["Longitude"] = 0

for index, row in df.iterrows():
    address = '{}, {}'.format(row["City"], row["Country"])

    geolocator = Nominatim()
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    
    df_cities.loc[index, "Latitude"] = latitude
    df_cities.loc[index, "Longitude"] = longitude
    print('The geograpical coordinate of {} City are {}, {}.'.format(row["City"], latitude, longitude))

/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  import sys


The geograpical coordinate of Tokyo  City are 35.6828387, 139.7594549.
The geograpical coordinate of New York  City are 40.7308619, -73.9871558.
The geograpical coordinate of Los Angeles  City are 34.0536834, -118.2427669.
The geograpical coordinate of London  City are 51.5073219, -0.1276474.
The geograpical coordinate of Paris  City are 48.8566101, 2.3514992.
The geograpical coordinate of Chicago  City are 41.8755616, -87.6244212.
The geograpical coordinate of Seoul  City are 37.5666791, 126.9782914.
The geograpical coordinate of Osaka City are 34.6937569, 135.5014539.
The geograpical coordinate of Sao Paulo City are -23.5506507, -46.6333824.
The geograpical coordinate of Rhine-Ruhr  City are 51.437302, 7.3345115.
The geograpical coordinate of Dallas City are 32.7762719, -96.7968559.
The geograpical coordinate of Washington, D.C. City are 38.8950092, -77.0365625.
The geograpical coordinate of Houston  City are 29.7589382, -95.3676974.
The geograpical coordinate of San Francisco  City 

In [20]:
df_cities

,City,Country,Region,GDP,Population,GDP_Div_Pop,Latitude,Longitude
0,Tokyo,Japan,East Asia,1893.000,38001000,49.814479,35.682839,139.759455
1,New York,United States,North America,1717.712,18593220,92.383783,40.730862,-73.987156
2,Los Angeles,United States,North America,1001.000,12309530,81.319108,34.053683,-118.242767
3,London,United Kingdom,Northern Europe,879.500,10313307,85.278175,51.507322,-0.127647
4,Paris,France,Western Europe,850.000,10843285,78.389529,48.856610,2.351499
5,Chicago,United States,North America,651.222,8744835,74.469330,41.875562,-87.624421
6,Seoul,South Korea,East Asia,635.400,9773746,65.010898,37.566679,126.978291
7,Osaka,Japan,East Asia,584.000,20237645,28.857113,34.693757,135.501454
8,Sao Paulo,Brazil,South America,582.079,21066245,27.630885,-23.550651,-46.633382
9,Rhine-Ruhr,Germany,Western Europe,573.700,10356889,55.393082,51.437302,7.334511


## 2. Explore Venues in Tokyo

Explore venues in Tokyo

In [32]:
# create map of Boroughs in Toronto using latitude and longitude values
map_tokyo = folium.Map(location=[df_cities.loc[0,"Latitude"], df_cities.loc[0,"Longitude"]], zoom_start=11)

# add markers to map
lat = df_cities.loc[0]['Latitude']
lng = df_cities.loc[0]['Longitude']
label = df_cities.loc[0]['City']

label = folium.Popup(label, parse_html=True)
folium.CircleMarker(
    [lat, lng],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_tokyo)  
    
map_tokyo

Before we get the data and start exploring it, let's download all the dependencies that we will need.

#### Define Foursquare Credentials and Version

In [36]:
CLIENT_ID = '3ES0INYJN3CE5SBJGBTUNGWY0OQM30YD11UMKKFXJTG2P1XP' # your Foursquare ID
CLIENT_SECRET = 'BGQ1TEAYASKLBAKAC3YCO15KRW4RT1AFRLBATI3XJT00JPLE' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 3ES0INYJN3CE5SBJGBTUNGWY0OQM30YD11UMKKFXJTG2P1XP
CLIENT_SECRET:BGQ1TEAYASKLBAKAC3YCO15KRW4RT1AFRLBATI3XJT00JPLE


In [34]:
city_latitude = df_cities.loc[0, 'Latitude'] # tokyo city latitude value
city_longitude = df_cities.loc[0, 'Longitude'] # tkyo longitude value

city_name = df_cities.loc[0, 'City'] # City name

print('Latitude and longitude values of {} are {}, {}.'.format(city_name, 
                                                               city_latitude, 
                                                               city_longitude))

Latitude and longitude values of Tokyo  are 35.6828387, 139.7594549.


#### Now, let's get the top 100 venues that are in Tokyo within a radius of 5000 meters.

In [37]:
radius = 5000
LIMIT = 100
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(\
    CLIENT_ID, CLIENT_SECRET, city_latitude, city_longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=3ES0INYJN3CE5SBJGBTUNGWY0OQM30YD11UMKKFXJTG2P1XP&client_secret=BGQ1TEAYASKLBAKAC3YCO15KRW4RT1AFRLBATI3XJT00JPLE&ll=35.6828387,139.7594549&v=20180605&radius=5000&limit=100'

In [38]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c33518a4434b96ea016519d'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Tokyo',
  'headerFullLocation': 'Tokyo',
  'headerLocationGranularity': 'city',
  'totalResults': 239,
  'suggestedBounds': {'ne': {'lat': 35.72783874500004,
    'lng': 139.81475261285402},
   'sw': {'lat': 35.63783865499995, 'lng': 139.704157187146}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b05879af964a520ac9b22e3',
       'name': 'Palace Hotel Tokyo (パレスホテル東京)',
       'location': {'address': '丸の内1-1-1',
        'lat': 35.684644,
        'lng': 139.761302,
        'labeledLatLngs': [{'label': 'display',
          'lat': 35.684644,
          'lng': 139.761302}]

define get_category_type() function to get the category type of the venue

In [39]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [40]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Palace Hotel Tokyo (パレスホテル東京),Hotel,35.684644,139.761302
1,Imperial Palace East Garden (皇居東御苑),Garden,35.685797,139.756662
2,Wolfgang's Steakhouse,Steakhouse,35.679278,139.762148
3,Aman Tokyo (アマン東京),Hotel,35.685515,139.765384
4,Mitsubishi Ichigokan Museum (三菱一号館美術館),Art Museum,35.678420,139.763260


In [43]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))
print('{} categories were returned by Foursquare.'.format(len(nearby_venues.categories.unique())))

100 venues were returned by Foursquare.
52 categories were returned by Foursquare.


In [45]:
nearby_venues.groupby("categories").count().sort_values('name', ascending=False)

,name,lat,lng
categories,,,
Hotel,8,8,8
Ramen Restaurant,5,5,5
Coffee Shop,5,5,5
Soba Restaurant,5,5,5
Japanese Curry Restaurant,4,4,4
Japanese Restaurant,4,4,4
Electronics Store,3,3,3
Yoshoku Restaurant,3,3,3
Sushi Restaurant,3,3,3


## 4. Explore Venues in all Cities

#### Let's create a function to repeat the same process to all Cities

In [47]:
def getNearbyVenues(names, latitudes, longitudes, radius=5000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['City', 
                  'City Latitude', 
                  'City Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [48]:
cities_venues = getNearbyVenues(names=df_cities["City"], 
                                   latitudes=df_cities["Latitude"], 
                                   longitudes=df_cities["Longitude"])

Tokyo 
New York 
Los Angeles 
London 
Paris 
Chicago 
Seoul 
Osaka
Sao Paulo
Rhine-Ruhr 
Dallas
Washington, D.C.
Houston 
San Francisco 
Shanghai 
Beijing 
Shenzhen


#### Let's check the size of the resulting dataframe

In [49]:
print(cities_venues.shape)
cities_venues.head()

(1659, 7)


,City,City Latitude,City Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Tokyo,35.682839,139.759455,Palace Hotel Tokyo (パレスホテル東京),35.684644,139.761302,Hotel
1,Tokyo,35.682839,139.759455,Imperial Palace East Garden (皇居東御苑),35.685797,139.756662,Garden
2,Tokyo,35.682839,139.759455,Wolfgang's Steakhouse,35.679278,139.762148,Steakhouse
3,Tokyo,35.682839,139.759455,Aman Tokyo (アマン東京),35.685515,139.765384,Hotel
4,Tokyo,35.682839,139.759455,Mitsubishi Ichigokan Museum (三菱一号館美術館),35.678420,139.763260,Art Museum


Let's check how many venues were returned for each city

In [50]:
cities_venues.groupby('City').count()

,City Latitude,City Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
City,,,,,,
Beijing,100,100,100,100,100,100
Chicago,100,100,100,100,100,100
Dallas,100,100,100,100,100,100
Houston,100,100,100,100,100,100
London,100,100,100,100,100,100
Los Angeles,100,100,100,100,100,100
New York,100,100,100,100,100,100
Osaka,100,100,100,100,100,100
Paris,100,100,100,100,100,100


In [51]:
print('There are {} uniques categories.'.format(len(cities_venues['Venue Category'].unique())))
print('There are {} uniques Venues.'.format(len(cities_venues['Venue'].unique())))

There are 272 uniques categories.
There are 1598 uniques Venues.


## 5. Analyze the Feature for Each City

In [53]:
# one hot encoding
cities_onehot = pd.get_dummies(cities_venues[['Venue Category']], prefix="", prefix_sep="")

# add borough column back to dataframe
cities_onehot['City'] = cities_venues['City'] 

# move city column to the first column
fixed_columns = [cities_onehot.columns[-1]] + list(cities_onehot.columns[:-1])
cities_onehot = cities_onehot[fixed_columns]

cities_onehot.head()

,City,Adult Boutique,American Restaurant,Amphitheater,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Baiano Restaurant,Bakery,Bar,Baseball Stadium,Basketball Stadium,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Beijing Restaurant,Big Box Store,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Churrascaria,Clothing Store,Cocktail Bar,Coffee Shop,College Quad,Comedy Club,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cultural Center,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dive Bar,Dog Run,Donburi Restaurant,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Flower Shop,Food Court,Food Truck,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gukbap Restaurant,Gym,Gym / Fitness Center,Gym Pool,Hainan Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Historic Site,History Museum,Hobby Shop,Home Service,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Karaoke Bar,Kebab Restaurant,Korean Restaurant,Kushikatsu Restaurant,Lake,Latin American Restaurant,Library,Liquor Store,Lounge,Mac & Cheese Joint,Marijuana Dispensary,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Mountain,Movie Theater,Multiplex,Museum,Music Venue,Nabe Restaurant,Neighborhood,New American Restaurant,Nightclub,Nightlife Spot,Noodle House,Okonomiyaki Restaurant,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Palace,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Peking Duck Restaurant,Performing Arts Venue,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pie Shop,Pizza Place,Planetarium,Plaza,Poke Place,Portuguese Restaurant,Pub,Public Art,Ramen Restaurant,Record Shop,Recording Studio,Resort,Restaurant,Rock Club,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shopping Mall,Shopping Plaza,Shrine,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soba Restaurant,Soccer Stadium,Social Club,South Indian Restaurant,Southern / Soul Food Restaurant,Souvenir Shop,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Spiritual Center,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Sukiyaki Restaurant,Supermarket,Sushi Restaurant,Synagogue,Szechuan Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Takoyaki Place,Tapas Restaurant,Tea Room,Tech Startup,Temple,Tempura Restaurant,Thai Restaurant,Theater,Tonkatsu Restaurant,Toy / Game Store,Track,Trail,Train Station,Trattoria/Osteria,Turkish Restaurant,Udon Restaurant,Unagi Restaurant,Used Bookstore,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Village,Wagashi Place,Waterfront,Wine Bar,Wine Shop,Xinjiang Restaurant,Yakitori Restaurant,Yoga Studio,Yoshoku Restaurant,Yunnan Restaurant,Zhejiang Restaurant,Zoo
0,Tokyo,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [55]:
cities_onehot.shape

(1659, 273)

#### Next, let's group rows by borough and by taking the mean of the frequency of occurrence of each category

In [56]:
cities_grouped = cities_onehot.groupby('City').mean().reset_index()
cities_grouped

,City,Adult Boutique,American Restaurant,Amphitheater,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Baiano Restaurant,Bakery,Bar,Baseball Stadium,Basketball Stadium,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Beijing Restaurant,Big Box Store,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Churrascaria,Clothing Store,Cocktail Bar,Coffee Shop,College Quad,Comedy Club,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cultural Center,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dive Bar,Dog Run,Donburi Restaurant,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Flower Shop,Food Court,Food Truck,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gukbap Restaurant,Gym,Gym / Fitness Center,Gym Pool,Hainan Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Historic Site,History Museum,Hobby Shop,Home Service,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Karaoke Bar,Kebab Restaurant,Korean Restaurant,Kushikatsu Restaurant,Lake,Latin American Restaurant,Library,Liquor Store,Lounge,Mac & Cheese Joint,Marijuana Dispensary,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Mountain,Movie Theater,Multiplex,Museum,Music Venue,Nabe Restaurant,Neighborhood,New American Restaurant,Nightclub,Nightlife Spot,Noodle House,Okonomiyaki Restaurant,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Palace,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Peking Duck Restaurant,Performing Arts Venue,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pie Shop,Pizza Place,Planetarium,Plaza,Poke Place,Portuguese Restaurant,Pub,Public Art,Ramen Restaurant,Record Shop,Recording Studio,Resort,Restaurant,Rock Club,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shopping Mall,Shopping Plaza,Shrine,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soba Restaurant,Soccer Stadium,Social Club,South Indian Restaurant,Southern / Soul Food Restaurant,Souvenir Shop,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Spiritual Center,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Sukiyaki Restaurant,Supermarket,Sushi Restaurant,Synagogue,Szechuan Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Takoyaki Place,Tapas Restaurant,Tea Room,Tech Startup,Temple,Tempura Restaurant,Thai Restaurant,Theater,Tonkatsu Restaurant,Toy / Game Store,Track,Trail,Train Station,Trattoria/Osteria,Turkish Restaurant,Udon Restaurant,Unagi Restaurant,Used Bookstore,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Village,Wagashi Place,Waterfront,Wine Bar,Wine Shop,Xinjiang Restaurant,Yakitori Restaurant,Yoga Studio,Yoshoku Restaurant,Yunnan Restaurant,Zhejiang Restaurant,Zoo
0,Beijing,0.00,0.00,0.00,0.00,0.00,0.0

In [57]:
cities_grouped.shape

(17, 273)

In [58]:
num_top_venues = 10

for city in cities_grouped['City']:
    print("----"+city+"----")
    temp = cities_grouped[cities_grouped['City'] == city].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Beijing ----
                    venue  freq
0           Historic Site  0.18
1                   Hotel  0.13
2                    Park  0.08
3                  Hostel  0.04
4             Coffee Shop  0.04
5  Peking Duck Restaurant  0.03
6       Yunnan Restaurant  0.03
7           Shopping Mall  0.03
8      Chinese Restaurant  0.03
9                    Café  0.03


----Chicago ----
                     venue  freq
0                  Theater  0.07
1                    Hotel  0.05
2                     Park  0.05
3               Donut Shop  0.03
4       Italian Restaurant  0.03
5                      Bar  0.03
6       Seafood Restaurant  0.03
7  New American Restaurant  0.03
8               Steakhouse  0.03
9                Gastropub  0.03


----Dallas----
                     venue  freq
0              Coffee Shop  0.06
1                      Bar  0.05
2                    Hotel  0.05
3                     Park  0.04
4       Mexican Restaurant  0.04
5  New American Restaurant  0.04
6

First, let's write a function to sort the venues in descending order.

In [59]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [61]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['City']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
cities_venues_sorted = pd.DataFrame(columns=columns)
cities_venues_sorted['City'] = cities_grouped['City']

for ind in np.arange(cities_grouped.shape[0]):
    cities_venues_sorted.iloc[ind, 1:] = return_most_common_venues(cities_grouped.iloc[ind, :], num_top_venues)

cities_venues_sorted

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Beijing,Historic Site,Hotel,Park,Hostel,Coffee Shop,Chinese Restaurant,Yunnan Restaurant,Café,Peking Duck Restaurant,Shopping Mall
1,Chicago,Theater,Park,Hotel,Seafood Restaurant,Gastropub,Donut Shop,Bar,Italian Restaurant,Steakhouse,New American Restaurant
2,Dallas,Coffee Shop,Hotel,Bar,Mexican Restaurant,Burger Joint,New American Restaurant,Park,Dive Bar,Cocktail Bar,Japanese Restaurant
3,Houston,Vietnamese Restaurant,Park,Bar,Pizza Place,Beer Garden,Cocktail Bar,Hotel,Mexican Restaurant,Baseball Stadium,American Restaurant
4,London,Hotel,Theater,Indian Restaurant,Steakhouse,Plaza,Seafood Restaurant,Bookstore,Park,Boutique,Pizza Place
5,Los Angeles,Ice Cream Shop,Art Gallery,Sushi Restaurant,Coffee Shop,Bar,Music Venue,Theater,Bookstore,Italian Restaurant,Ramen Restaurant
6,New York,Park,Italian Restaurant,Sushi Restaurant,Seafood Restaurant,Hotel,Pizza Place,Bakery,Bookstore,Gourmet Shop,Thai Restaurant
7,Osaka,Japanese Curry Restaurant,Sake Bar,Ramen Restaurant,Hotel,Bakery,Coffee Shop,Japanese Restaurant,Kushikatsu Restaurant,Bar,BBQ Joint
8,Paris,French Restaurant,Plaza,Cocktail Bar,Sandwich Place,Bookstore,Wine Bar,Art Museum,Historic Site,Hotel,Italian Restaurant
9,Rhine-Ruhr,Supermarket,Drugstore,Fast Food Restaurant,Café,Bakery,Hardware Store,Big Box Store,Doner Restaurant,Ice Cream Shop,Furniture / Home Store


In [63]:
# set number of clusters
kclusters = 5

cities_grouped_clustering = cities_grouped.drop('City', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(cities_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([1, 0, 0, 0, 2, 0, 0, 2, 2, 3, 0, 0, 2, 4, 4, 2, 2], dtype=int32)

In [64]:
cities_grouped.shape

(17, 273)

In [66]:
cities_merged = df_cities

# add clustering labels
cities_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
cities_merged = cities_merged.join(cities_venues_sorted.set_index('City'), on='City')

cities_merged # check the last columns!

,City,Country,Region,GDP,Population,GDP_Div_Pop,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Tokyo,Japan,East Asia,1893.000,38001000,49.814479,35.682839,139.759455,1,Hotel,Ramen Restaurant,Soba Restaurant,Coffee Shop,Japanese Restaurant,Japanese Curry Restaurant,Sushi Restaurant,Udon Restaurant,Tonkatsu Restaurant,Electronics Store
1,New York,United States,North America,1717.712,18593220,92.383783,40.730862,-73.987156,0,Park,Italian Restaurant,Sushi Restaurant,Seafood Restaurant,Hotel,Pizza Place,Bakery,Bookstore,Gourmet Shop,Thai Restaurant
2,Los Angeles,United States,North America,1001.000,12309530,81.319108,34.053683,-118.242767,0,Ice Cream Shop,Art Gallery,Sushi Restaurant,Coffee Shop,Bar,Music Venue,Theater,Bookstore,Italian Restaurant,Ramen Restaurant
3,London,United Kingdom,Northern Europe,879.500,10313307,85.278175,51.507322,-0.127647,0,Hotel,Theater,Indian Restaurant,Steakhouse,Plaza,Seafood Restaurant,Bookstore,Park,Boutique,Pizza Place
4,Paris,France,Western Europe,850.000,10843285,78.389529,48.856610,2.351499,2,French Restaurant,Plaza,Cocktail Bar,Sandwich Place,Bookstore,Wine Bar,Art Museum,Historic Site,Hotel,Italian Restaurant
5,Chicago,United States,North America,651.222,8744835,74.469330,41.875562,-87.624421,0,Theater,Park,Hotel,Seafood Restaurant,Gastropub,Donut Shop,Bar,Italian Restaurant,Steakhouse,New American Restaurant
6,Seoul,South Korea,East Asia,635.400,9773746,65.010898,37.566679,126.978291,0,Korean Restaurant,Hotel,Historic Site,Café,Noodle House,Palace,Japanese Restaurant,Art Museum,Park,Plaza
7,Osaka,Japan,East Asia,584.000,20237645,28.857113,34.693757,135.501454,2,Japanese Curry Restaurant,Sake Bar,Ramen Restaurant,Hotel,Bakery,Coffee Shop,Japanese Restaurant,Kushikatsu Restaurant,Bar,BBQ Joint
8,Sao Paulo,Brazil,South America,582.079,21066245,27.630885,-23.550651,-46.633382,2,Theater,Cultural Center,Pizza Place,Brazilian Restaurant,Bookstore,Italian Restaurant,Ice Cream Shop,Art Museum,Park,Café
9,Rhine-Ruhr,Germany,Western Europe,573.700,10356889,55.393082,51.437302,7.334511,3,Supermarket,Drugstore,Fast Food Restaurant,Café,Bakery,Hardware Store,Big Box Store,Doner Restaurant,Ice Cream Shop,Furniture / Home Store


## 6. Examine the Similarity Matrix

In [136]:
from scipy import spatial


Compute the similarities among cities

In [133]:
cities_features = cities_grouped.set_index('City')
cities_names = list(cities_features.index)
# cities_features.head()
len(cities_features.index)
cities_similarity = pd.DataFrame(columns=list(cities_features.index))

for row_city in cities_names:
    row_city_vector = []
    list1 = list(cities_features.loc[row_city])
    for column_city in cities_names:
        list2 = list(cities_features.loc[column_city])
        sim = 1 - spatial.distance.cosine(list1, list2)
        row_city_vector.append(sim)
    cities_similarity.loc[row_city] = row_city_vector

cities_similarity.head()

,Beijing,Chicago,Dallas,Houston,London,Los Angeles,New York,Osaka,Paris,Rhine-Ruhr,San Francisco,Sao Paulo,Seoul,Shanghai,Shenzhen,Tokyo,"Washington, D.C."
Beijing,1.000000,0.335841,0.346995,0.280185,0.440741,0.181007,0.243969,0.267581,0.294181,0.096601,0.231381,0.155560,0.542747,0.527140,0.534757,0.373914,0.284104
Chicago,0.335841,1.000000,0.591727,0.491465,0.583646,0.453926,0.488558,0.246369,0.258478,0.083356,0.390950,0.434749,0.313003,0.394508,0.409542,0.331169,0.422592
Dallas,0.346995,0.591727,1.000000,0.637698,0.516346,0.460086,0.355295,0.333441,0.304897,0.073744,0.489314,0.340662,0.310289,0.350514,0.484985,0.365184,0.487772
Houston,0.280185,0.491465,0.637698,1.000000,0.379337,0.441013,0.382124,0.221701,0.265700,0.082973,0.350842,0.327292,0.245846,0.257205,0.328512,0.219620,0.330468
London,0.440741,0.583646,0.516346,0.379337,1.000000,0.340425,0.446815,0.383746,0.447341,0.134473,0.347969,0.442232,0.399596,0.607814,0.497935,0.400741,0.506690


Find most similar city for cities

In [150]:
sim_dic = {}
sim_df = pd.DataFrame(columns=["most similar city"])

for city_row in cities_names:
    max_val = 0
    for city_column in cities_names:
        if city_row != city_column:
            if cities_similarity.loc[city_row][city_column] > max_val:
                max_val = cities_similarity.loc[city_row][city_column]
                sim_dic[city_row] = city_column
                sim_df.loc[city_row] = city_column
sim_df

,most similar city
Beijing,Seoul
Chicago,Dallas
Dallas,Houston
Houston,Dallas
London,Shanghai
Los Angeles,Sao Paulo
New York,San Francisco
Osaka,Tokyo
Paris,London
Rhine-Ruhr,Shenzhen


## 7. Examine Clusters

Examine each cluster and determine the discriminating venue categories that distinguish each cluster.

#### Cluster 1

In [140]:
cities_merged.loc[cities_merged['Cluster Labels'] == 0, cities_merged.columns[[0] + list(range(8, cities_merged.shape[1]))]]

,City,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,New York,0,Park,Italian Restaurant,Sushi Restaurant,Seafood Restaurant,Hotel,Pizza Place,Bakery,Bookstore,Gourmet Shop,Thai Restaurant
2,Los Angeles,0,Ice Cream Shop,Art Gallery,Sushi Restaurant,Coffee Shop,Bar,Music Venue,Theater,Bookstore,Italian Restaurant,Ramen Restaurant
3,London,0,Hotel,Theater,Indian Restaurant,Steakhouse,Plaza,Seafood Restaurant,Bookstore,Park,Boutique,Pizza Place
5,Chicago,0,Theater,Park,Hotel,Seafood Restaurant,Gastropub,Donut Shop,Bar,Italian Restaurant,Steakhouse,New American Restaurant
6,Seoul,0,Korean Restaurant,Hotel,Historic Site,Café,Noodle House,Palace,Japanese Restaurant,Art Museum,Park,Plaza
10,Dallas,0,Coffee Shop,Hotel,Bar,Mexican Restaurant,Burger Joint,New American Restaurant,Park,Dive Bar,Cocktail Bar,Japanese Restaurant
11,"Washington, D.C.",0,Monument / Landmark,Art Museum,Hotel,History Museum,Coffee Shop,Salon / Barbershop,American Restaurant,Indian Restaurant,Theater,Science Museum


Keywords: Park, Restaurant

#### Cluster 2

In [141]:
cities_merged.loc[cities_merged['Cluster Labels'] == 1, cities_merged.columns[[0] + list(range(8, cities_merged.shape[1]))]]

,City,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Tokyo,1,Hotel,Ramen Restaurant,Soba Restaurant,Coffee Shop,Japanese Restaurant,Japanese Curry Restaurant,Sushi Restaurant,Udon Restaurant,Tonkatsu Restaurant,Electronics Store


Keywords: Hotel, Restaurant

#### Cluster 3

In [142]:
cities_merged.loc[cities_merged['Cluster Labels'] == 2, cities_merged.columns[[0] + list(range(8, cities_merged.shape[1]))]]

,City,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Paris,2,French Restaurant,Plaza,Cocktail Bar,Sandwich Place,Bookstore,Wine Bar,Art Museum,Historic Site,Hotel,Italian Restaurant
7,Osaka,2,Japanese Curry Restaurant,Sake Bar,Ramen Restaurant,Hotel,Bakery,Coffee Shop,Japanese Restaurant,Kushikatsu Restaurant,Bar,BBQ Joint
8,Sao Paulo,2,Theater,Cultural Center,Pizza Place,Brazilian Restaurant,Bookstore,Italian Restaurant,Ice Cream Shop,Art Museum,Park,Café
12,Houston,2,Vietnamese Restaurant,Park,Bar,Pizza Place,Beer Garden,Cocktail Bar,Hotel,Mexican Restaurant,Baseball Stadium,American Restaurant
15,Beijing,2,Historic Site,Hotel,Park,Hostel,Coffee Shop,Chinese Restaurant,Yunnan Restaurant,Café,Peking Duck Restaurant,Shopping Mall
16,Shenzhen,2,Hotel,Coffee Shop,Café,Chinese Restaurant,Shopping Mall,Nightclub,Electronics Store,Spa,Restaurant,Japanese Restaurant


Keywords: Restaurant, Hotel

#### Cluster 4

In [144]:
cities_merged.loc[cities_merged['Cluster Labels'] == 3, cities_merged.columns[[0] + list(range(8, cities_merged.shape[1]))]]

,City,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,Rhine-Ruhr,3,Supermarket,Drugstore,Fast Food Restaurant,Café,Bakery,Hardware Store,Big Box Store,Doner Restaurant,Ice Cream Shop,Furniture / Home Store


Keywords: Store, Supermarket, Resturant

#### Cluster 5


In [143]:
cities_merged.loc[cities_merged['Cluster Labels'] == 4, cities_merged.columns[[0] + list(range(8, cities_merged.shape[1]))]]

,City,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,San Francisco,4,Coffee Shop,Grocery Store,Yoga Studio,Bakery,Pizza Place,Sushi Restaurant,Ice Cream Shop,Park,Art Museum,Gastropub
14,Shanghai,4,Hotel,Hotel Bar,Lounge,Chinese Restaurant,French Restaurant,Scenic Lookout,Dumpling Restaurant,Café,Restaurant,Italian Restaurant


Keywords: Resturant, Coffe